Задача из отбора на стажировку в АНО «ЦИСМ» 2021.01.20

Выполнил Коротков Илья

**У меня не получилось запустить итоговый файл с решением на платформе Kaggle, так как я потратил всё время графического ускорителя. Прикладываю ссылку на решение в Google Colab, в котором есть выводы всех ячеек.**

Ссылка на решение в Google Colab - https://colab.research.google.com/drive/1lb54XXjysEqBCSjeIXR7dN0s6MWMffbv?usp=sharing

Ссылка на видео с описанием решения - https://drive.google.com/file/d/1Ky1aMFGfz34Wb3eWeXvs2KgxjgGYtwk3/view?usp=sharing

In [ ]:
from os import listdir
from os.path import join, isfile, isdir
from PIL import Image

import torch
from torch.utils.data import Dataset, SubsetRandomSampler
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import image
%matplotlib inline

device = torch.device('cuda:0')

# 1. Постановка задачи

В данной задаче вам необходимо классифицировать футбольные команды по их форме. Обучающая выборка состоит 3 408 фото, разбитых на 10 футбольных команд: Arsenal, Chelsea, Liverpool, Manchester City, Manchester United, Real Madrid, Barcelona, Bayern Munich, Paris Saint-Germain, Juventus.
В тестовой выборке 1 177 фото. Пример сабмита содержится в файле sample_submission.csv. Лейблы в сабмите должны быть закодированы следующим образом:

Arsenal: 0, Barcelona: 1, Bayern: 2, Chelsea: 3, Juventus: 4, Liverpool: 5, ManchesterCity: 6, ManchesterUnited: 7, PSG: 8, Real: 9

Вам необходимо построить модель нейронной сети для классификации футбольной формы. В качестве метрики выбрана F1 Macro.

# 2. Загрузка данных

Загрузим и распакуем заранее подготовленные данные с Google диска.

In [ ]:
!pip install gdown

In [ ]:
!gdown --id 1cnZ7__tXkH7cZjOEiZuKGzF4CVKFgiSt -O "/content/train.zip"
!gdown --id 1oyqP49cgJG5oxIBdiHMQoNsHonCCPasf -O "/content/test.zip"

In [ ]:
%%capture
!unzip "/content/train.zip" -d "/content/"
!unzip "/content/test.zip" -d "/content/"

# 3. Предобработка данных

Напишем свой класс FootballTeamsDataset для загрузки данных и работы с ними в PyTorch.

In [ ]:
class FootballTeamsDataset(Dataset):
  teams_dict = {
      'Arsenal': 0,
      'Barcelona': 1,
      'Bayern': 2,
      'Chelsea': 3,
      'Juventus': 4,
      'Liverpool': 5,
      'ManchesterCity': 6,
      'ManchesterUnited': 7,
      'PSG': 8,
      'Real': 9,
      '': -1,
  }

  def __init__(self, folder, transform=None):
    self.transform = transform
    self.folder = folder
    self.files = []
    self.teams = []

    for file in listdir(folder):
      if isdir(join(folder, file)):
        # Set label as file's folder name
        team_folder = join(folder, file)
        self.files.extend([f for f in listdir(team_folder) if isfile(join(team_folder, f))])
        self.teams.extend([file] * len(listdir(team_folder)))
      elif isfile(join(folder, file)):
        # If file is not inside folder, set empty label
        self.files.append(file)
        self.teams.append('')
      
  def __len__(self):
    return len(self.files)
  
  def __getitem__(self, index):
    if torch.is_tensor(index):
      index = index.tolist()

    img_name = self.files[index]
    img_path = join(self.folder, self.teams[index], img_name)

    img_label = self.teams_dict[self.teams[index]]

    img = Image.open(img_path)
    
    if self.transform is not None:
      img = self.transform(img)

    return img, img_label, img_path

In [ ]:
train_folder = '/content/train'
test_folder = '/content/test'

orig_dataset = FootballTeamsDataset(train_folder)

Убедимся, что в тренировочном датасете отсутствует дисбаланс классов.

In [ ]:
labels = []
for i in range(0, len(orig_dataset)):
  labels.append(orig_dataset[i][1])

pd.Series(labels).hist()
plt.title('Количество изображений по классам');

Посмотрим на несколько изображений из датасета.

In [ ]:
def visualize_samples(dataset, indices):
  count = len(indices)
  plt.figure(figsize=(count*4, 4))
  plt.suptitle(f'{count} random samples')

  for i, index in enumerate(indices):
    x, y, _ = dataset[index]
    x = np.array(x)

    if len(x.shape) == 3:
      x = x[0]

    plt.subplot(1,count,i+1)
    plt.title(f'Label: {y}')
    plt.imshow(x)
    plt.grid(False)
    plt.axis('off')

In [ ]:
random_indices = np.random.choice(np.arange(len(orig_dataset)), 8, replace=False)

visualize_samples(orig_dataset, random_indices)

Посмотрим на распределение разрешения изображений в датасете.

In [ ]:
indices = np.arange(len(orig_dataset))
df_sizes = pd.DataFrame({'width': [], 'height': []})
for index in indices:
  x, y, _ = orig_dataset[index]
  width = x.size[0]
  height = x.size[1]

  df_sizes = df_sizes.append(pd.DataFrame({
      'width': [width],
      'height': [height],
  }))
df_sizes = df_sizes.reset_index(drop=True)

In [ ]:
df_sizes['width'].hist()
plt.title('Распределение ширины изображений');

In [ ]:
df_sizes['height'].hist()
plt.title('Распределение высоты изображений');

Большинство изображений имеет ширину и высоту около 1024 пикселей.

Создадим объекты датасетов для тренировочной и тестовой части.

Изменим разрешение входных изображений на 1024x1024, преобразуем их в тензоры и нормализуем. Для тренировочного датасета так же добавим аугментацию.

In [ ]:
train_transform = transforms.Compose([
  transforms.Resize((1024, 1024)),
  transforms.ColorJitter(brightness=.5, saturation=.5, hue=.3),
  transforms.RandomRotation(30, interpolation=transforms.InterpolationMode.BILINEAR),
  transforms.ToTensor(),
  transforms.Normalize((0.5), (0.5)),
])

test_transform = transforms.Compose([
  transforms.Resize((1024, 1024)),
  transforms.ToTensor(),
  transforms.Normalize((0.5), (0.5)),
])

train_dataset = FootballTeamsDataset(train_folder, transform=train_transform)
test_dataset = FootballTeamsDataset(test_folder, transform=test_transform)

Посмотрим на изображения с аугментацией.

In [ ]:
visualize_samples(train_dataset, random_indices)

Разобъём тренировочный датасет на тренировочную и валидационную части.

Создадим загрузчики для тренировочных, валидационных и тестовых данных.

In [ ]:
np.random.seed(69)

batch_size = 8

# Get data indices
data_size = len(train_dataset)
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

# Create train and validation data loaders
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         sampler=val_sampler)

# Create test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

# 4. Создание модели

Создадим модель нейросети с четырьмя свёрточными слоями и тремя полносвязными. Добавим batch normalization и dropout для регуляризации.

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, 5, padding=2)
    self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
    self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
    self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
    self.pool = nn.MaxPool2d(4)
    self.batchnorm16 = nn.BatchNorm2d(16)
    self.batchnorm32 = nn.BatchNorm2d(32)
    self.batchnorm64 = nn.BatchNorm2d(64)
    self.dropout = nn.Dropout(p=0.2)
    self.fc1 = nn.Linear(128 * 4 * 4, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 10)
  
  def forward(self, x):
    x = self.batchnorm16(self.pool(F.relu(self.conv1(x))))
    x = self.batchnorm32(self.pool(F.relu(self.conv2(x))))
    x = self.batchnorm64(self.pool(F.relu(self.conv3(x))))
    x = self.pool(F.relu(self.conv4(x)))
    x = torch.flatten(x, 1)
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.fc3(x)
    return x

# 5. Обучение модели

Функция обучения модели принимает на вход саму модель, загрузчики тренировочных и валидационных данных, loss функцию, оптимизатор, scheduler (опционально) и количество эпох для обучения.

После обучения каждой эпохи рассчитывается метрика для тренировочных и валидационных данных, она выводятся в консоль, а обученная модель сохраняется на диск.

In [ ]:
def train_model(model, train_loader, val_loader, loss, optimizer, scheduler, num_epochs):
  best_val = 1
  loss_history = []
  train_history = []
  val_history = []

  for epoch in range(num_epochs):
    model.train() # Enter train mode
    
    loss_accum = 0
    predicted_labels = []
    real_labels = []
    for x, y, _ in tqdm(train_loader):
      # Move data to GPU
      inputs, labels = x.to(device), y.to(device)
      # Make prediction
      prediction = model(inputs)
      # Calculate loss
      loss_value = loss(prediction, labels)
      # Optimize loss function
      optimizer.zero_grad()
      loss_value.backward()
      optimizer.step()
      # Save real and predicted labels to calculate score
      _, predicted = torch.max(prediction, 1)
      predicted_labels.extend(predicted.tolist())
      real_labels.extend(labels.tolist())

      # Clear GPU memeory
      del inputs
      del labels

      loss_accum += loss_value

    ave_loss = loss_accum / len(train_loader)
    train_score = f1_score(real_labels, predicted_labels, average='macro')
    val_score = compute_f1(model, val_loader)
    
    loss_history.append(float(ave_loss))
    train_history.append(train_score)
    val_history.append(val_score)

    if scheduler:
      scheduler.step()

    if val_score < best_val:
      best_val = val_score
      torch.save(model.state_dict(), '/content/models/best_model.pt')
    torch.save(model.state_dict(), '/content/models/model.pt')
    
    print(f'Epoch: {epoch+1}/{num_epochs}, Average loss: {ave_loss}, Train score: {train_score}, Val score: {val_score}')
      
  return loss_history, train_history, val_history

def compute_f1(model, loader):
    model.eval() # Evaluation mode

    predicted_labels = []
    real_labels = []
    for x, y, _ in tqdm(loader):
      # Move data to GPU
      inputs, labels = x.to(device), y.to(device)
      # Make prediction
      prediction = model(inputs)
      # Save real and predicted labels to calculate score
      _, predicted = torch.max(prediction, 1)
      predicted_labels.extend(predicted.tolist())
      real_labels.extend(labels.tolist())

      # Clear GPU memeory
      del inputs
      del labels
    
    score = f1_score(real_labels, predicted_labels, average='macro')
    return score

Инициализируем модель и перенесём её на GPU.

Также создадим объекты для функции потерь и оптимизатора Adam с параметрами скорости обучения $lr=3*10^{-4}$ и коэффициента регуляризации $weight\_decay=10^{-4}$


In [ ]:
model = Net().to(device)

loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-4)
#scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2, eta_min=1e-6)

Запустим обучение модели на 30 эпох.

In [ ]:
%%time
loss_history, train_history, val_history = train_model(model, train_loader, val_loader, loss, optimizer, None, 30)

Сохраним полученную модель на диск.

In [ ]:
torch.save(model.state_dict(), '/content/models/model_30ep.pt')

# 6. Анализ обученной модели

Визуализируем графики функции потерь и метрики.

In [ ]:
plt.plot(loss_history)

plt.title('Train loss');

In [ ]:
plt.plot(train_history)
plt.plot(val_history)

plt.legend(['Train', 'Validation'])
plt.title('F1 score');

Посмотрим на ошибки модели.

In [ ]:
df_pred = pd.DataFrame({'path': [], 'real_label': [], 'pred_label': []})

model.eval() # Evaluation mode
for x, y, path in tqdm(val_loader):
  # Predict the label
  inputs = x.to(device)
  prediction = model(inputs)
  _, predicted = torch.max(prediction.data, 1)

  df_pred = df_pred.append(pd.DataFrame({
      'path': path,
      'real_label': y,
      'pred_label': predicted.cpu().detach().numpy(),
  }))
df_pred['real_label'] = df_pred['real_label'].astype(int)
df_pred['pred_label'] = df_pred['pred_label'].astype(int)
df_pred = df_pred.reset_index(drop=True)

In [ ]:
df_errors = df_pred[df_pred['real_label'] != df_pred['pred_label']].reset_index(drop=True)
sns.heatmap(confusion_matrix(df_errors['real_label'], df_errors['pred_label']));

Какой-то связи между командами и ошибками модели не видно. Чаще всего модель ошибается классифицируя форму Barcelona как форму Juventus.

Посмотрим на сами изображения, на которых ошибается модель.

In [ ]:
fig = plt.figure(figsize=(20, 20))
cols = 5
rows = 4
random_indices = np.random.choice(np.arange(df_errors.shape[0]), cols * rows, replace=False)
for n, index in enumerate(random_indices):
  fig.add_subplot(rows, cols, n+1)
  plt.imshow(Image.open(df_errors.iloc[index]['path']))
  plt.grid(False)
  plt.axis('off')

# 7. Предсказания модели

Функция generate_submission_file генерирует файл с предсказаниями модели на тестовых данных.

In [ ]:
def generate_sumbission_file(model, test_loader, file_name='submission'):
  submission_df = pd.DataFrame({'img_name': [], 'label': []})

  model.eval() # Evaluation mode
  for x, y, path in tqdm(test_loader):
    # Predict the label
    inputs = x.to(device)
    prediction = model(inputs)
    _, predicted = torch.max(prediction.data, 1)

    # Save prediction
    img_name = path[0]
    img_name = img_name[img_name.rfind('/')+1:]
    submission_df = submission_df.append(pd.DataFrame({
        'img_name': [img_name],
        'label': [predicted.item()]
    }))
  # Convert labels from float to int
  submission_df['label'] = submission_df['label'].astype(int)
  submission_df = submission_df.reset_index(drop=True)

  # Save submissions dataframe to file
  path = f'/content/submissions/{file_name}.csv'
  submission_df.to_csv(path, index=False)

  print(f'\nSubmission file saved to: {path}')

Сохраним предсказания модели.

In [ ]:
generate_sumbission_file(model, test_loader, 'submission1')